In [1]:
# xgb_gui_fixed_colorful.py
import tkinter as tk
from tkinter import ttk, messagebox
import os
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor



In [2]:
# === User setting ===
EXCEL_PATH = "Dataset_Jayed_GUI.xlsx"

XGB_PARAMS = dict(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=5,
    min_child_weight=3,
    subsample=0.6,
    colsample_bytree=0.95,
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=42,
)

# === Mapping: Excel column headers → Display names ===
PARAM_NAMES = {
    "teff/D": "Effective thickness ratio",
    "a/d": "Shear span-to-depth ratio",
    "Prestressed reinforcement index (MPa)": "Prestressed reinforcement index (MPa)",
    "Non-prestressed reinforcement index (MPa)": "Non-prestressed reinforcement index (MPa)",
    "Shear reinforcement index (MPa)": "Shear reinforcement index (MPa)",
    "Concrete grade, (MPa)": "Concrete compressive strength (MPa)",
    "Tota axial stress(MPa)": "Total axial stress (MPa)",
    "Maximum flexural stress(MPa)": "Maximum flexural stress (MPa)",
}




In [26]:
class XGBGuiApp:
    def __init__(self, root):
        self.root = root
        self.root.title("XGBoost GUI – Flexural Stress Predictor")
        # for windows, fixed size
        # self.root.geometry("1300x700")
        # for windows, max size
        self.root.state("zoomed")
        # for linux/mac
        #self.root.attributes("-zoomed", True)
        self._palette()
        self.root.configure(bg=self.COLORS["bg"])

        # Fonts
        self.FONT = ("Times New Roman", 12)
        self.FONT_BOLD = ("Times New Roman", 13, "bold")
        self.FONT_TITLE = ("Times New Roman", 16, "bold")

        self.feature_names = []
        self.target_name = None
        self.model = None
        self.input_vars = {}
        self.predict_var = tk.StringVar(value="—")
        self.status_var = tk.StringVar(value="Load a dataset to begin…")

        self._style_config()
        self._build_ui()

    # --- Color palette ---
    def _palette(self):
        self.COLORS = {
            "bg":        "#F7F7FB",  # app background
            "surface":   "#FFFFFF",  # cards
            "accent":    "#4C78FF",  # header / primary
            "accent_lt": "#6D90FF",
            "accent_dk": "#2C56E8",
            "muted":     "#A0AEC0",
            "text":      "#1F2937",
            "ok":        "#06B178",
            "ok_dk":     "#059669",
            "danger":    "#EF4444",
            "danger_dk": "#DC2626",
            "warning":   "#F59E0B",
        }

    # === ttk Theme + Font Styling ===
    def _style_config(self):
        style = ttk.Style()
        style.theme_use("clam")

        # Frames / Cards
        style.configure("TFrame", background=self.COLORS["bg"])
        style.configure("Card.TLabelframe",
                        background=self.COLORS["surface"],
                        borderwidth=0, padding=18)
        style.configure("Card.TLabelframe.Label",
                        background=self.COLORS["surface"],
                        foreground=self.COLORS["text"],
                        font=self.FONT_BOLD)

        # Labels, entries
        style.configure("TLabel", background=self.COLORS["surface"],
                        foreground=self.COLORS["text"], font=self.FONT)
        style.configure("Field.TLabel", background=self.COLORS["bg"],
                        foreground=self.COLORS["text"], font=self.FONT)

        style.configure("Field.TEntry",
                        fieldbackground="#FFFFFF",
                        bordercolor=self.COLORS["muted"],
                        lightcolor=self.COLORS["accent_lt"],
                        darkcolor=self.COLORS["muted"],
                        relief="flat",
                        padding=6)
        style.map("Field.TEntry",
                  fieldbackground=[("focus", "#FFFDF2")])

        # Buttons
        style.configure("Primary.TButton",
                        background=self.COLORS["ok"],
                        foreground="#FFFFFF",
                        padding=8, font=self.FONT_BOLD, borderwidth=0)
        style.map("Primary.TButton",
                  background=[("active", self.COLORS["ok_dk"])])
        style.configure("Danger.TButton",
                        background=self.COLORS["danger"],
                        foreground="#FFFFFF",
                        padding=8, font=self.FONT_BOLD, borderwidth=0)
        style.map("Danger.TButton",
                  background=[("active", self.COLORS["danger_dk"])])

        # Progressbar
        style.configure("Accent.Horizontal.TProgressbar",
                        troughcolor="#E8EDFF", background=self.COLORS["accent"])

    # === Build Main UI ===
    def _build_ui(self):
        # Header bar
        header = tk.Frame(self.root, bg=self.COLORS["accent"], height=58)
        header.pack(fill=tk.X, side=tk.TOP)
        tk.Label(header, text="Flexural Stress Predictor",
                 bg=self.COLORS["accent"], fg="#FFFFFF",
                 font=self.FONT_TITLE).pack(side=tk.LEFT, padx=18, pady=12)
        tk.Label(header, text="XGBoost · StandardScaler",
                 bg=self.COLORS["accent"], fg="#DDE6FF",
                 font=("Times New Roman", 12)).pack(side=tk.RIGHT, padx=18)

        # Main area
        main = ttk.Frame(self.root, padding=18, style="TFrame")
        main.pack(fill=tk.BOTH, expand=True)

        # LEFT: Inputs (card)
        left_wrapper = ttk.Labelframe(main, text="Inputs", style="Card.TLabelframe")
        left_wrapper.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.form_container = ttk.Frame(left_wrapper, style="TFrame")
        self.form_container.pack(fill=tk.BOTH, expand=True)

        # RIGHT: Prediction & Info (two stacked cards)
        right_col = ttk.Frame(main, style="TFrame")
        right_col.pack(side=tk.LEFT, fill=tk.Y, padx=(18, 0))

        # Prediction card
        out_box = ttk.Labelframe(right_col, text="Prediction", style="Card.TLabelframe")
        out_box.pack(fill=tk.X, pady=(0, 18))

        ttk.Label(out_box, text="Predicted Maximum Flexural Stress (MPa):",
                  font=self.FONT_BOLD).grid(row=0, column=0, sticky="w", pady=(0, 8))
        ttk.Entry(out_box, textvariable=self.predict_var, state="readonly",
                  width=22, style="Field.TEntry").grid(row=1, column=0, pady=(0, 10), sticky="w")

        self.pb = ttk.Progressbar(out_box, mode="indeterminate",
                                  length=220, style="Accent.Horizontal.TProgressbar")
        self.pb.grid(row=2, column=0, pady=(0, 8), sticky="w")

        ttk.Button(out_box, text="Predict", style="Primary.TButton",
                   command=self.predict, width=18).grid(row=3, column=0, pady=(6, 6), sticky="w")
        ttk.Button(out_box, text="Clear", style="Danger.TButton",
                   command=self.clear_inputs, width=18).grid(row=4, column=0, pady=(0, 2), sticky="w")

        # Info card
        info = ttk.Labelframe(right_col, text="Info", style="Card.TLabelframe")
        info.pack(fill=tk.BOTH, expand=True)
        self.info_label = ttk.Label(info, textvariable=self.status_var, wraplength=260, justify="left")
        self.info_label.pack(anchor="w")

        # claude code for images
        image_frame = tk.Frame(self.root, bg=self.COLORS["bg"], height=120)
        image_frame.pack(fill=tk.X, side=tk.BOTTOM, padx=18, pady=(0, 10))

        tk.Label(image_frame, 
                text="Summary plot and global importance of the input features",
                bg=self.COLORS["bg"], 
                fg=self.COLORS["text"],
                font=self.FONT_BOLD).pack(pady=(0, 10))
        
        try:
            # Load and display first image
            from PIL import Image, ImageTk
            img1 = Image.open("image1.jpg")  # Replace with your image path
            # img1 = img1.resize((200, 100))  # Adjust size as needed
            self.photo1 = ImageTk.PhotoImage(img1)
            tk.Label(image_frame, image=self.photo1, bg=self.COLORS["bg"]).pack(side=tk.LEFT, padx=10)
            
            # Load and display second image
            img2 = Image.open("image2.jpg")  # Replace with your image path
            # img2 = img2.resize((200, 100))  # Adjust size as needed
            self.photo2 = ImageTk.PhotoImage(img2)
            tk.Label(image_frame, image=self.photo2, bg=self.COLORS["bg"]).pack(side=tk.LEFT, padx=10)
        except Exception as e:
            tk.Label(image_frame, text=f"Images not loaded: {e}", 
                    bg=self.COLORS["bg"], fg=self.COLORS["muted"]).pack()

        # Footer/status
        footer = tk.Frame(self.root, bg=self.COLORS["surface"], height=28)
        footer.pack(fill=tk.X, side=tk.BOTTOM)
        self.footer_msg = tk.Label(footer, text="Ready", bg=self.COLORS["surface"],
                                   fg=self.COLORS["muted"], font=("Times New Roman", 11))
        self.footer_msg.pack(side=tk.LEFT, padx=12)

    # === Load Excel and Train Model ===
    def load_excel_from_path(self, path):
        if not os.path.exists(path):
            messagebox.showerror("File Not Found", f"Excel file not found:\n{path}")
            return

        df = pd.read_excel(path)
        self.feature_names = list(df.columns[:-1])
        self.target_name = df.columns[-1]

        X = df[self.feature_names].values
        y = df[self.target_name].values

        self.model = Pipeline([
            ("scaler", StandardScaler()),
            ("xgb", XGBRegressor(**XGB_PARAMS))
        ])
        self.model.fit(X, y)

        self._build_form()
        self.status_var.set(
            f"Loaded dataset: {os.path.basename(path)}  •  "
            f"Features: {len(self.feature_names)}  •  Target: {self.target_name}"
        )
        self.footer_msg.config(text="Model trained")

    # === Build Input Form ===
    def _build_form(self):
        for widget in self.form_container.winfo_children():
            widget.destroy()

        # create rows
        self.input_vars = {}
        for i, name in enumerate(self.feature_names):
            label_name = PARAM_NAMES.get(name, name)

            # alternating row background effect using nested frames
            row_frame = tk.Frame(self.form_container,
                                 bg=self.COLORS["bg"] if i % 2 else self.COLORS["surface"])
            row_frame.grid(row=i, column=0, sticky="ew", padx=0, pady=2)
            row_frame.grid_columnconfigure(1, weight=1)

            ttk.Label(row_frame, text=label_name, style="Field.TLabel")\
                .grid(row=0, column=0, sticky="w", padx=(2, 8), pady=6)

            var = tk.StringVar()
            self.input_vars[name] = var
            ttk.Entry(row_frame, textvariable=var, width=26, style="Field.TEntry")\
                .grid(row=0, column=1, sticky="w", padx=(0, 4), pady=6)

    # === Predict Button Action ===
    def predict(self):
        # show subtle animation
        try:
            self.pb.start(12)
            self.footer_msg.config(text="Predicting…")
            self.root.after(150, self._do_predict)  # slight delay for visual feedback
        except Exception as e:
            messagebox.showerror("Prediction Error", str(e))
            self.pb.stop()
            self.footer_msg.config(text="Prediction failed")

    def _do_predict(self):
        try:
            values = [float(self.input_vars[n].get()) for n in self.feature_names]
            X_new = np.array(values).reshape(1, -1)
            y_pred = self.model.predict(X_new)
            self.predict_var.set(f"{y_pred[0]:.4f}")
            self.footer_msg.config(text="Prediction complete")
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter numeric values in all fields.")
            self.footer_msg.config(text="Invalid inputs")
        except Exception as e:
            messagebox.showerror("Prediction Error", str(e))
            self.footer_msg.config(text="Prediction failed")
        finally:
            self.pb.stop()

    # === Clear Button Action ===
    def clear_inputs(self):
        for v in self.input_vars.values():
            v.set("")
        self.predict_var.set("—")
        self.footer_msg.config(text="Cleared")




In [27]:
def main():
    root = tk.Tk()
    app = XGBGuiApp(root)
    app.load_excel_from_path(EXCEL_PATH)
    root.mainloop()

if __name__ == "__main__":
    main()